<!-- Projeto Desenvolvido na Universidade Federal do Tocantins -->
# Universidade Federal do Tocantins
## Inteligência Artificial Para Visão Computacional
## Projeto 2
### Fine-Tuning de Modelo Pré-Treinado Para Classificação de Imagens de Animais Silvestres

## Instalando e Carregando Pacotes

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3

In [ ]:
# Imports
import numpy as np
import torch
import torchvision
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from datasets import load_metric
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%reload_ext watermark
%watermark -a "Universidade Federal do Tocantins"

In [ ]:
dataset = 'onepiece'

## Automatizando a Carga dos Seus Próprios Dados Para Ajuste do Modelo

In [ ]:
# Carrega o dataset no formato zip e descompacta
# dados = load_dataset("imagefolder", data_files = f'data/{dataset}.zip')
# Carrega o dataset diretamente do Hugging Face Hub

dados = load_dataset("BangumiBase/onepiece")

In [ ]:
print(dados)

In [ ]:
type(dados)

## Explorando os Dados

In [ ]:
# Temos imagens e labels no dicionário dados
dados["train"].features

In [ ]:
# Detalhes da imagem de índice 1142 (por exemplo)
print(dados['train'][1142])

In [ ]:
# Extraindo a imagem de índice 1142
imagem = dados["train"][1142]

In [ ]:
# Cada imagem tem a matriz de pixels no formato PIL e o label
imagem

In [ ]:
# Imprimindo o label
imagem['label']

In [ ]:
# Visualizamos a imagem
imagem['image']

In [ ]:
# Testando o redimensionamento da imagem usando o método resize
imagem['image'].resize((800, 400))

O campo `label` não é um rótulo no formato de string. Por padrão, os campos `ClassLabel` são codificados em números inteiros por conveniência, no pacote datasets. Mas podemos extrair o nome de classe assim:

In [ ]:
# Nomes dos labels
dados["train"].features['label'].names

In [ ]:
# Label de índice 1
dados["train"].features['label'].names[1]

## Criando Mapeamentos Índice/Nome de Classe

Vamos criar um dicionário chamado `id2label` para decodificar os ids de classes em strings. O `label2id` inverso também será útil quando carregarmos o modelo posteriormente.

In [ ]:
# Extrai os nomes de labels
labels = dados["train"].features["label"].names

In [ ]:
# Gera os objetos para os mapeamentos
label2id, id2label = dict(), dict()

In [ ]:
# Loop carregar os mapeamentos
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
id2label

In [ ]:
label2id

## Pré-Processamento das Imagens

https://huggingface.co/google/vit-base-patch16-224

In [ ]:
# Nome do repositório no HF
dsa_modelo_hf = "google/vit-base-patch16-224"

In [ ]:
# Import do processador de imagens usado no treinamento do modelo no HF
dsa_image_processor = AutoImageProcessor.from_pretrained(dsa_modelo_hf)

In [ ]:
dsa_image_processor

Aqui definimos 2 funções separadas, uma para treinamento (que inclui aumento de dados) e outra para validação (que inclui apenas redimensionamento, corte central e normalização).

In [ ]:
# Normalização das imagens
dsa_normalize = Normalize(mean = dsa_image_processor.image_mean, std = dsa_image_processor.image_std)

In [ ]:
type(dsa_normalize)

Vamos extrair alguns detalhes do processador de imagens e usar isso ao preparar as transformações para nossas próprias imagens.

In [ ]:
# Verifica se a chave 'height' está presente no dicionário 'size' do objeto 'dsa_image_processor'
if "height" in dsa_image_processor.size:

    # Se 'height' está presente, define 'size' como uma tupla contendo a altura e largura
    size = (dsa_image_processor.size["height"], dsa_image_processor.size["width"])

    # Define 'crop_size' igual ao 'size' definido anteriormente
    crop_size = size

    # Define 'max_size' como None, pois não é especificado neste ramo da condição
    max_size = None

# Verifica se a chave 'shortest_edge' está presente no dicionário 'size' do objeto 'dsa_image_processor'
elif "shortest_edge" in dsa_image_processor.size:

    # Se 'shortest_edge' está presente, define 'size' como o valor de 'shortest_edge'
    size = dsa_image_processor.size["shortest_edge"]

    # Define 'crop_size' como uma tupla com ambos os valores sendo 'size'
    crop_size = (size, size)

    # Define 'max_size' como o valor de 'longest_edge' ou None se 'longest_edge' não existir
    max_size = dsa_image_processor.size.get("longest_edge")

In [ ]:
# Cria a composição das transformações nos dados de treino
transformacoes_treino = Compose([RandomResizedCrop(crop_size), RandomHorizontalFlip(), ToTensor(), dsa_normalize])

In [ ]:
# Cria a composição das transformações nos dados de validação/teste
transformacoes_valid = Compose([Resize(size), CenterCrop(crop_size), ToTensor(), dsa_normalize])

In [ ]:
# Função de pré-processamento de dados de treino
def dsa_preprocessa_treino(lote_dados):

    lote_dados["pixel_values"] = [transformacoes_treino(image.convert("RGB")) for image in lote_dados["image"]]

    return lote_dados

In [ ]:
# Função de pré-processamento de dados de validação/teste
def dsa_preprocessa_valid(lote_dados):

    lote_dados["pixel_values"] = [transformacoes_valid(image.convert("RGB")) for image in lote_dados["image"]]

    return lote_dados

A seguir, podemos pré-processar nosso conjunto de dados aplicando essas funções. Usaremos a funcionalidade `set_transform`, que permite aplicar as funções acima on-the-fly (ou seja, elas só serão aplicadas quando as imagens forem carregadas na memória RAM).

In [ ]:
# Vamos criar o índice para dividir os dados de treino em treino e validação
splits = dados["train"].train_test_split(test_size = 0.1)

In [ ]:
# Dados de treino
dados_treino = splits['train']

In [ ]:
# Aplica o pré-processamento
dados_treino.set_transform(dsa_preprocessa_treino)

In [ ]:
# Matriz de pixels e label da imagem de índice 10
dados_treino[10]

In [ ]:
# Dados de validação
dados_valid = splits['test']

In [ ]:
# Aplica o pré-processamento
dados_valid.set_transform(dsa_preprocessa_valid)

In [ ]:
# Matriz de pixels e label da imagem de índice 23
dados_valid[23]

## Definindo Argumentos e Hiperparâmetros do Fine-Tuning

Agora que nossos dados estão prontos, podemos baixar o modelo pré-treinado e ajustá-lo.

Para classificação usamos a classe `AutoModelForImageClassification`. Chamar o método `from_pretrained` fará o download e armazenará em cache os pesos do modelo.

Como os IDs dos rótulos e o número de rótulos dependem do conjunto de dados, passamos `label2id` e `id2label` junto com o repositório para download do modelo pré-treinado. Isso garantirá que um cabeçalho de classificação personalizado seja criado (com um número personalizado de neurônios de saída).

In [ ]:
# Carrega o modelo pré-treinado
modelo = AutoModelForImageClassification.from_pretrained(dsa_modelo_hf,
                                                         label2id = label2id,
                                                         id2label = id2label,
                                                         ignore_mismatched_sizes = True)

O aviso acima está nos dizendo que estamos descartando alguns pesos (os pesos e bias da camada `classificador`) e inicializando aleatoriamente alguns outros (os pesos e bias de uma nova camada `classificador`). Isso é esperado neste caso, porque estamos adicionando um novo cabeçote para o qual não temos pesos pré-treinados, então a biblioteca nos avisa que devemos ajustar esse modelo antes de usá-lo para inferência, que é exatamente o que vamos fazer.

In [ ]:
# Pasta para salvar o modelo
modelo_salvar = dsa_modelo_hf.split("/")[-1]

In [ ]:
# Hiperparâmetros
batch_size = 32
taxa_aprendizado = 5e-5
accumulation_steps = 4
num_epochs = 3
wratio = 0.1
lsteps = 10

Veja a descrição completa dos hiperparâmetros acima no Capítulo 6 do curso.

In [ ]:
# Argumentos de treino
dsa_args = TrainingArguments(f"{modelo_salvar}-dsa-p2-finetuned",
                             remove_unused_columns = False,
                             evaluation_strategy = "epoch",
                             save_strategy = "epoch",
                             learning_rate = taxa_aprendizado,
                             per_device_train_batch_size = batch_size,
                             gradient_accumulation_steps = accumulation_steps,
                             per_device_eval_batch_size = batch_size,
                             num_train_epochs = num_epochs,
                             warmup_ratio = wratio,
                             logging_steps = lsteps,
                             load_best_model_at_end = True,
                             metric_for_best_model = "accuracy",
                             push_to_hub = False)

A seguir, precisamos definir uma função para calcular as métricas das previsões, que usará apenas a `métrica` que carregamos anteriormente. O único pré-processamento que precisamos fazer é pegar o argmax dos nossos logits previstos.

Logits são os valores brutos de saída de uma camada de rede neural antes de serem normalizados por uma função de ativação, como a função softmax em problemas de classificação. Em termos mais técnicos, os logits são as entradas para a última função de ativação de uma rede neural, que é responsável por transformar esses valores brutos em probabilidades.

Para entender melhor, considere o contexto de uma rede neural usada para classificação. Na última camada da rede, antes da aplicação da função softmax, você tem um conjunto de valores, cada um correspondendo a uma classe potencial. Estes valores são os logits. Eles podem ser positivos, negativos, grandes ou pequenos, e não estão restritos a um intervalo específico.

A função softmax, então, é aplicada a esses logits para transformá-los em probabilidades. A softmax assegura que a soma das probabilidades de todas as classes seja igual a 1, tornando os valores mais interpretáveis e úteis para classificação. Cada logit é transformado em uma probabilidade que representa a confiança do modelo de que a entrada pertence à classe correspondente.

In [ ]:
# Métrica
dsa_metrica = load_metric("accuracy")

In [ ]:
# Função para calcular as métricas
def dsa_compute_metrics(eval_pred):

    # Previsões do modelo
    predictions = np.argmax(eval_pred.predictions, axis = 1)

    # Retorna a métrica
    return dsa_metrica.compute(predictions = predictions, references = eval_pred.label_ids)

Também definimos um `collate_fn`, que será usado para agrupar exemplos. Cada lote consiste em 2 chaves, sendo `pixel_values` e `labels`.

In [ ]:
# Definição de uma função de collate personalizada para o DataLoader
def dsa_collate_fn(examples):

    # Agrupa os valores dos pixels de cada exemplo em um batch, usando torch.stack
    pixel_values = torch.stack([example["pixel_values"] for example in examples])

    # Cria um tensor com as labels (etiquetas) de cada exemplo no batch
    labels = torch.tensor([example["label"] for example in examples])

    # Retorna um dicionário contendo os valores dos pixels e as labels correspondentes
    return {"pixel_values": pixel_values, "labels": labels}

> Agora só precisamos passar tudo isso junto com nossos conjuntos de dados para o `Trainer`!

In [ ]:
# Cria o Trainer
dsa_trainer = Trainer(modelo,
                      dsa_args,
                      train_dataset = dados_treino,
                      eval_dataset = dados_valid,
                      tokenizer = dsa_image_processor,
                      compute_metrics = dsa_compute_metrics,
                      data_collator = dsa_collate_fn)

## Treinamento do Modelo

Agora podemos ajustar nosso modelo chamando o método `train`.

In [ ]:
# %%time
# resultados_treino = dsa_trainer.train()

Conseguimos cerca de 94% de acurácia em apenas 18 minutos de treinamento de um incrível modelo de Visão Computacional.

In [ ]:
# # Salvamos modelo e métricas
# dsa_trainer.save_model()
# dsa_trainer.log_metrics("train", resultados_treino.metrics)
# dsa_trainer.save_metrics("train", resultados_treino.metrics)
# dsa_trainer.save_state()

In [ ]:
import os
from transformers import AutoModelForImageClassification

modelo_salvo_path = f"models/{dataset}-trained-model"

if os.path.exists(os.path.join(modelo_salvo_path, "pytorch_model.bin")) or os.path.exists(os.path.join(modelo_salvo_path, "model.safetensors")):
    print("Modelo já treinado encontrado. Carregando modelo salvo...")
    modelo = AutoModelForImageClassification.from_pretrained(modelo_salvo_path)
else:
    print("Modelo salvo não encontrado. Treinando modelo...")
    modelo = AutoModelForImageClassification.from_pretrained(
        dsa_modelo_hf,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True
    )
    dsa_trainer = Trainer(
        modelo,
        dsa_args,
        train_dataset=dados_treino,
        eval_dataset=dados_valid,
        tokenizer=dsa_image_processor,
        compute_metrics=dsa_compute_metrics,
        data_collator=dsa_collate_fn
    )
    resultados_treino = dsa_trainer.train()
    dsa_trainer.save_model()
    dsa_trainer.log_metrics("train", resultados_treino.metrics)
    dsa_trainer.save_metrics("train", resultados_treino.metrics)
    dsa_trainer.save_state()

## Avaliação do Modelo

Criamos o avaliador e extraímos as métricas de avaliação do modelo.

In [ ]:
import os
import json

# Caminho para os resultados de avaliação
avaliacao_salva_path = "models/oregon-wild-life-trained-model/eval_results.json"

if os.path.exists(avaliacao_salva_path):
    print("Resultados de avaliação encontrados. Carregando resultados salvos...")
    with open(avaliacao_salva_path, "r") as f:
        avaliador = json.load(f)
else:
    print("Resultados de avaliação não encontrados. Avaliando modelo...")
    avaliador = dsa_trainer.evaluate()
    with open(avaliacao_salva_path, "w") as f:
        json.dump(avaliador, f)

print(avaliador)

In [ ]:
# # Extrai as métricas com o avaliador
# dsa_trainer.log_metrics("eval", avaliador)
# dsa_trainer.save_metrics("eval", avaliador)

Conseguimos cerca de 94% de acurácia usando o avaliador.

## Usando o Modelo Para Previsões com Novas Imagens

In [ ]:
# Imports
from PIL import Image
from IPython.display import display

In [ ]:
# Carrega a imagem
image = Image.open('imagem01.jpeg')
#image = Image.open('imagem02.jpeg')

In [ ]:
type(image)

In [ ]:
from PIL import Image
from IPython.display import display

img_pil = Image.open('imagem01.jpeg')
img_pequena = img_pil.resize((224, 224))
display(img_pequena)

In [ ]:
# Preprocess the image
image = transformacoes_valid(image.convert("RGB")).unsqueeze(0)

In [ ]:
# Move image to the same device as the model
image = image.to(modelo.device)

In [ ]:
# Faz a previsão
with torch.no_grad():
    logits = modelo(image).logits

In [ ]:
print(logits)

In [ ]:
# Get predicted label
id_label_previsto = logits.argmax(-1).item()

In [ ]:
# Convert label id to label name
nome_label_previsto = id2label[id_label_previsto]

In [ ]:
print(f"Label Previsto: {nome_label_previsto}")

In [ ]:
%watermark -a "Universidade Federal do Tocantins"

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim